In [1]:
import ctypes
import subprocess
import sys
import os
import platform

In [2]:

# --- Configuration ---
# Default model to host. Llama 2 requires manual 'huggingface-cli login' inside WSL first.
# Using a non-gated model is recommended for first-time automated setup.
DEFAULT_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 
# --- End Configuration ---


In [3]:

def is_admin():
    """Checks if the script is running with Administrator privileges on Windows."""
    try:
        return ctypes.windll.shell32.IsUserAnAdmin() == 1
    except AttributeError:
        return False  # Not on Windows


In [4]:

def check_wsl_installed():
    """Checks if WSL is installed by running 'wsl.exe --list'."""
    print("Checking for WSL installation...")
    try:
        # Use '--list' and check return code.
        # '--quiet' is not used here to provide more context on failure.
        subprocess.run(
            ['wsl.exe', '--list', '--verbose'], 
            check=True, 
            capture_output=True, 
            text=True
        )
        print("...WSL installation detected.")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("...WSL not detected.")
        return False


In [5]:

def install_wsl():
    """Triggers the WSL installation command."""
    print("--- [STAGE 1] WSL Installation ---")
    print("Attempting to install WSL with Ubuntu distribution...")
    print("This will open a new console window for the installer.")
    try:
        # Run the installation in a new console window
        subprocess.run(
            ['wsl.exe', '--install', '-d', 'Ubuntu'], 
            check=True,
            # This flag helps show the installer progress in its own window
            creationflags=subprocess.CREATE_NEW_CONSOLE 
        )
        print("\n--- ACTION REQUIRED ---")
        print("1. WSL installation has been initiated.")
        print("2. Please REBOOT your computer to complete the installation.")
        print("3. After rebooting, a terminal may open to set up your Linux username and password. Please complete this.")
        print("4. After setup is complete, re-run this *same script* (run_petals_wsl.py) as Administrator.")
        print("------------------------")
        
    except subprocess.CalledProcessError as e:
        print(f"\nError: WSL installation failed (Code: {e.returncode}).")
        print("Please ensure virtualization is enabled in your BIOS and try again.")
    except FileNotFoundError:
        print("\nError: 'wsl.exe' command not found. Your Windows version may be too old.")
    
    sys.exit(0)


In [6]:
def run_petals_in_wsl(model_name):
    """Executes the full Petals server setup and launch chain inside WSL as root."""
    
    print("--- [STAGE 2] Petals Server Setup & Launch (inside WSL) ---")
    
    # Check for manual Hugging Face login requirement
    if "llama" in model_name.lower() or "mixtral" in model_name.lower():
        print("\n[!] WARNING: You have selected a gated model.")
        print("This automated script CANNOT bypass the Hugging Face login.")
        print("If you have not done so, this script will fail when downloading the model.")
        print("\n--- MANUAL ACTION RECOMMENDED (before proceeding) ---")
        print("1. Open 'cmd'.")
        print("2. Type 'wsl'.")
        print("3. Inside the Linux terminal, run: huggingface-cli login")
        print("4. Follow the prompts to log in.")
        print("5. Type 'exit' to close WSL and return here.")
        
        while True:
            confirm = input("Have you logged in to Hugging Face inside WSL? (yes/no): ").strip().lower()
            if confirm == 'yes':
                break
            elif confirm == 'no':
                print("Please complete the manual login step first.")
                sys.exit(1)

    print("\nBuilding setup command for WSL...")
    
    # This entire block is executed as a single command by 'bash -c'
    # It runs as root to avoid all 'sudo' password prompts.
    # Note: Using '\' to escape newlines in a Python f-string requires the newline
    # itself, which is why this is structured as a multi-line string.
    
    command_string = f"""
echo '--- [1/5] Updating apt packages (this may take a while)... ---' && \
apt-get update -y && \
apt-get upgrade -y && \
\
echo '--- [2/5] Installing Python 3, pip, and venv... ---' && \
apt-get install python3 python3-pip python3-venv -y && \
\
echo '--- [3/5] Creating Python virtual environment at /opt/petals-env... ---' && \
python3 -m venv /opt/petals-env && \
\
echo '--- [4.1/5] Installing Petals... ---' && \
/opt/petals-env/bin/pip install petals && \
\
echo '--- [4.2/5] Installing PyTorch (CPU version)... ---' && \
/opt/petals-env/bin/pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu && \
\
echo '--- [5/5] All dependencies installed. Launching Petals server... ---' && \
echo '--- To stop the server, press Ctrl+C in this window. ---' && \
/opt/petals-env/bin/python -m petals.cli.run_server {model_name}
"""

    print("Starting setup and server launch inside WSL.")
    print("This will take a significant amount of time.")
    print("The output from WSL will stream below:")
    print("---------------------------------------------------------")
    
    try:
        # Execute the entire chain using 'wsl.exe -u root'
        subprocess.run(
            ['wsl.exe', '-u', 'root', '-e', 'bash', '-c', command_string],
            check=True
        )
    except subprocess.CalledProcessError as e:
        print(f"\n--- ERROR ---")
        print(f"WSL setup failed with return code {e.returncode}.")
        print("Please review the output above for errors.")
        print("Common issues:")
        print(" - You did not log in to Hugging Face for a gated model.")
        print(" - An 'apt' or 'pip' package failed to install (network issue?).")
        print(f" - The model '{model_name}' does not exist or is not supported.")
    except KeyboardInterrupt:
        print("\n--- Server terminated by user (Ctrl+C) ---")


In [7]:
if platform.system() != "Windows":
    print("Error: This script is designed for Windows 11 with WSL.")
    sys.exit(1)


In [8]:

if not is_admin():
    print("Error: This script requires Administrator privileges to manage WSL.")
    print("Please re-run this script as Administrator.")
    sys.exit(1)

print("--- Torbiz Petals Seeder Setup Script ---")
    


--- Torbiz Petals Seeder Setup Script ---


In [9]:
if check_wsl_installed():
    # STAGE 2: WSL is present, run the setup.
    model = input(f"Enter the Hugging Face model name to host [{DEFAULT_MODEL}]: ")
    if not model:
        model = DEFAULT_MODEL
    
    run_petals_in_wsl(model.strip())
else:
    # STAGE 1: WSL is not present, install it.
    install_wsl()

Checking for WSL installation...
...WSL installation detected.
--- [STAGE 2] Petals Server Setup & Launch (inside WSL) ---

[!] WARNING: You have selected a gated model.
This automated script CANNOT bypass the Hugging Face login.
If you have not done so, this script will fail when downloading the model.

--- MANUAL ACTION RECOMMENDED (before proceeding) ---
1. Open 'cmd'.
2. Type 'wsl'.
3. Inside the Linux terminal, run: huggingface-cli login
4. Follow the prompts to log in.
5. Type 'exit' to close WSL and return here.

Building setup command for WSL...
Starting setup and server launch inside WSL.
This will take a significant amount of time.
The output from WSL will stream below:
---------------------------------------------------------

--- ERROR ---
WSL setup failed with return code 1.
Please review the output above for errors.
Common issues:
 - You did not log in to Hugging Face for a gated model.
 - An 'apt' or 'pip' package failed to install (network issue?).
 - The model 'TinyLla